## Description

This demo notebook guides you through sending metadata from edge AI devices (e.g. AGX Xavier) to Kafka server in parallel with DeepStream pipeline.

## Prerequisites

Follow deepstream-kafka.ipynb notebook to set up Kafka server

## 1. Compile the source code

In [ ]:
!cd ../../sources/apps/sample_apps/deepstream-test4/ && export CUDA_VER=11.1 && make

## 2. Run DeepStream to produce event messages

In [ ]:
!cd ../../sources/apps/sample_apps/deepstream-test4/ && \
    ./deepstream-test4-app -i /opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.h264 -p /opt/nvidia/deepstream/deepstream/lib/libnvds_kafka_proto.so  --conn-str="<your-host-ip>;9092" --topic="<your-created-topic>" -s 0

## 3. Consume event messages

In [ ]:
%%writefile ../../kafka/consumer.py
import time
from kafka import KafkaConsumer
from json import loads
import uuid

consumer = KafkaConsumer(
    '<your-created-topic>',
    bootstrap_servers='<your-host-ip>:9092',
    auto_offset_reset='latest',
    enable_auto_commit=True,
    group_id=str(uuid.uuid1()),
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

# do a dummy poll to retrieve some message
consumer.poll()

# go to end of the stream
consumer.seek_to_end()

for event in consumer:
    event_data = event.value
    print(event_data)

In [ ]:
!pip install kafka-python
!python3 ../../kafka/consumer.py